<a href="https://colab.research.google.com/github/mg-2321/Suicide-Attempt-and-Suicide-Ideation-detector/blob/main/SI_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import numpy as np
import pandas as pd


import os
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import transformers
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
raw = pd.read_csv('/content/drive/MyDrive/CG_NUS_RP/Suicide_Detection.csv')

In [ ]:
df = raw.sample(10000 , random_state=29)

In [ ]:
df= df.drop('Unnamed: 0',axis=1)

In [ ]:
df['class'].replace({'suicide' : 1 , 'non-suicide' : 0}, inplace=True)

In [ ]:
df

,text,class
213108,Nobody cares...Got to see / talk to my mom for...,1
214134,Lost and confusedSo this whole bout of depress...,1
136003,My life is falling apart and I don't know want...,1
192568,Mental BarrierHonestly I've stopped trying to ...,1
198908,I went ghost to focus on myself for a week and...,0
...,...,...
161542,I'm going through a lot stuff right now I'm go...,0
11809,I dont think I can keep goingmy ex and I recen...,1
175643,Bruh the girls that pull up their shorts and t...,0
69299,Anyone wanna chat rn ? Just vibin pm me if u w...,0


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
encoded = tokenizer(df['text'].tolist(), truncation=True, padding=True)
encoded_dataset = tf.data.Dataset.from_tensor_slices((encoded, df['class'].tolist()))

In [ ]:
encoded_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:
BATCH_SIZE = 2

def order(inp, label):

    return {
        'input_ids': inp['input_ids'],
        'attention_mask': inp['attention_mask'],
        'token_type_ids': inp['token_type_ids']
    }, label



train_size = int(0.8 * len(encoded_dataset))
val_size = int(0.1 * len(encoded_dataset))

train_dataset = encoded_dataset.take(train_size).map(order)
val_dataset = encoded_dataset.skip(train_size).take(val_size).map(order)
test_dataset = encoded_dataset.skip(train_size + val_size).map(order)


train_dataset = train_dataset.map(order)
val_dataset = val_dataset.map(order)
test_dataset = test_dataset.map(order)


train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset=test_dataset.batch(BATCH_SIZE)
train_dataset.shuffle.batch(BATCH_SIZE).shiffle(1000)
train_size=int(0.8*len(encoded_datset))

In [ ]:
text, label = next(iter(train_dataset))
print(text, '\n\n', label)

{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[ 101, 2385, 2546, ...,    0,    0,    0],
       [ 101, 1045, 2074, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>} 

 tf.Tensor([0 0], shape=(2,), dtype=int32)


In [ ]:
bert_model = TFAutoModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'bert_model': self.bert,
            'num_classes': self.fc.units,
        })
        return config


In [ ]:
model = BERTForClassification(bert_model)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_dataset,
    epochs=2,
    validation_data=val_dataset
)

Epoch 1/2
4000/4000 [==============================] - 60901s 15s/step - loss: 0.1785 - accuracy: 0.9333 - val_loss: 0.0977 - val_accuracy: 0.9720
Epoch 2/2
1614/4000 [===========>..................] - ETA: 9:45:48 - loss: 0.0756 - accuracy: 0.9768

In [ ]:
model.evaluate(test_dataset)

500/500 [==============================] - 48s 96ms/step - loss: 0.0911 - accuracy: 0.9610


[0.09112849831581116, 0.9610000252723694]

In [ ]:
y_true = []
for batch in test_dataset:
    x, y = batch
    y_true.extend(y.numpy())

y_pred_prob = model.predict(test_dataset)
y_pred = np.round(y_pred_prob).astype(int)

cm = confusion_matrix(y_true, y_pred)
print(cm)

500/500 [==============================] - 50s 95ms/step
[[489  24]
 [ 15 472]]


In [ ]:
test_m = ["Today I felt good in the morning, everything was good, but in the evening, it rained, and as a result, I got stuck in the traffic; my life sucks",
           "Today, I felt good in the morning; everything was good, but in the evening, it rained, and as a result, I got stuck in traffic. My life sucks; I should end it; I should kill myself."]

inputs = tokenizer(test_m ,padding=True, truncation=True,
                  return_tensors='tf')
model.call(inputs)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00209417],
       [0.9963147 ]], dtype=float32)>

In [ ]:
test_m = [" my life does not  sucks, i want to end it",
           "my life sucks, but i cannot end it"]

inputs = tokenizer(test_m ,padding=True, truncation=True,
                  return_tensors='tf')
model.call(inputs)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.96636957],
       [0.99055743]], dtype=float32)>

In [ ]:
# model.save('')